In [1]:
import openai
import json
import random
import re
import pandas as pd
from tqdm import tqdm
import os
from openai import OpenAI

api_key = "your openai api key"

In [2]:
client = OpenAI(api_key=api_key)

In [ ]:
# import dataset
#The work title corresponding to the character name
with open('profiles-eng_scripts.json', 'r', encoding='utf-8') as file:
    info = json.load(file)
#character description
with open('profiles-eng_desc.json', 'r', encoding='utf-8') as file:
    desp = json.load(file)

#Generate Big 5 personality traits analysis
#prompt_template = ("Based on the Big Five Personality Traits, analyze the character {character} from {work} in terms of Openness to Experience, Conscientiousness, Extraversion, Agreeableness, and Neuroticism.")
#prompt_template =("Analysing {character} in {work} based on the Big Five personality traits on the dimensions of Penetrance to Experience, Consientiousness, Extraversion, Agreeableness and Neuroticism")
prompt_template =("You are {character} in {work}. Here are some descriptions of you:{description}. Based on the Big Five Personality Traits, analyze yourself in terms of Openness to Experience, Conscientiousness, Extraversion, Agreeableness, and Neuroticism")
answers = {}

for character, work in info.items():
    description = desp[character]
    prompt = prompt_template.format(character=character, work=work, description = description)

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo", 
            messages=[{"role": "user", "content": prompt}],
        )
        answer = response.choices[0].message.content
        answers[character] = answer
    except Exception as e:
        print(f"Error processing character {character} from {work}: {e}")
    

with open('withdbig5.json', 'w', encoding='utf-8') as file:
    json.dump(answers, file, indent=4, ensure_ascii=False)

print(f"saved")

In [25]:
#get samples from the 95 characters
with open('big5.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

keys = list(data.keys())

num_keys_to_extract = int(len(keys) * 0.2)

sampled_keys = random.sample(keys, num_keys_to_extract)

print(sampled_keys)

['Sonny', 'Antonio Salieri', 'Professor G.H. Dorr', 'Tom Ripley', 'Po', 'Colonel Nathan R. Jessep', 'Freddy Krueger', 'Peter Parker', 'Violet Weston', 'David Aames', 'Doctor Who', 'Lestat de Lioncourt', 'Tugg Speedman', 'James Bond', 'Stephen Hawking', 'Juno MacGuff', 'Judge Dredd', 'Abraham Lincoln', 'Michael Scott']


In [54]:
import json

#check the big5 personality traits analysis
with open('newbig5.json', 'r', encoding='utf-8') as file:
    data1 = json.load(file)

print(data1['Frank T.J. Mackey'])

Frank T.J. Mackey, portrayed by Tom Cruise in the film Magnolia, can be analyzed based on the Big Five personality traits as follows:

1. Openness to Experience: Frank Mackey exhibits a high level of openness to experience. He is unapologetically bold, daring, and unconventional in his approach to life and relationships. He is not afraid to speak his mind and challenge societal norms. His willingness to take risks and pursue his goals with passion demonstrates his openness to new experiences.

2. Conscientiousness: Frank Mackey demonstrates high conscientiousness in his self-discipline and determination to succeed. He is a highly organized and driven individual who meticulously plans his seduction seminars and career. His ability to stay focused on his goals and work hard to achieve them reflects his conscientious nature.

3. Extraversion: Frank Mackey is a highly extraverted individual who thrives in social situations and seeks attention and validation from others. He is charismatic, 

In [10]:
#template to construct character personal profile
profile = {
    "target role": """description""",
    "related role 1": """description""",
    "related role 2": """description"""    
}

with open('.\profile\character.json', 'w', encoding='utf-8') as json_file:
    json.dump(profile, json_file, ensure_ascii=False, indent=4)

print("saved")

saved


In [4]:
#prompt for relationship questions
def generate_prompt(role_a, name, desp, profile):
    script_name = name[role_a]
    role_a_desc = profile[role_a]
    description = desp[role_a]
    num = 10
    prompt = f"""
System Instruction:
{role_a} in the {script_name} is a key figure in the story, with complex relationships with several important characters. Please generate {num} questions for each important character that reflect the changes in their relationship with Character A. To help you design the questions better, I will provide a brief description of {role_a} and brief descriptions of the important characters in the work.
The rules for designing questions are as follows:
1. Remember, all your questions should be directed towards the character {role_a}.{role_a} is the intended recipient of your questions.
2. Questions need to be complete. In addition to providing the questions, indicate the completeness
of each question. The completeness depends on whether the question specifies particular characters or events.
3. Questions should revolve around the main plot of the description.
4. Keep in mind that you need to design a total of {num} questions.
5. Based on the provided descriptions, you can still design questions based on your general knowledge of {script_name} and {role_a}.
Next, I will provide you with 4 examples. These examples are unrelated to {role_a} and
{script_name}. Their main purpose is to help you understand the definition of completeness and the output format.
[Examples]
Question 1: Tony Stark, who is your best friend in Iron Man?
Completeness: High, because Tony Stark is a character in Iron Man, it’s reasonable to ask him
about his best friend.
Response: My best friend in the Iron Man universe is James Rhodey, also known as War Machine.
We’ve been through a lot together, both in and out of our suits. Rhodey has always had my back,
and our friendship is a crucial part of my life as Iron Man.
Question 2: Tony Stark, why are you angry about Spider-Man’s behavior?
Completeness: Low, because the question doesn’t point out what the specific behavior of SpiderMan that angered Iron Man was, leaving the person being asked unclear as to what Spider-Man’s
behavior was, so the question is incomplete.
Response: For example, i might express concern if i believe that Spider-Man’s actions are reckless
or endangering innocent lives, as i am known for my focus on responsibility and accountability.
...
Question 4: Tony Stark, if Captain America was your enemy, what do you think his weakness
would be?
Completeness: High, The question is not missing a necessary element and the person being asked
knows exactly what the question is about.
Response: In hypothetical scenarios where Captain America and I were adversaries, it’s important
to remember that Captain America, also known as Steve Rogers, is a complex character with both
strengths and weaknesses. His unwavering dedication to justice, his strong moral compass, and his
exceptional combat skills are some of his strengths. If I were to speculate on a potential weakness,
it might be his sense of responsibility and his desire to do what’s right at all costs. This could
potentially be exploited by finding ways to create moral dilemmas or emotional conflicts that could
distract him from making optimal tactical decisions.
User Prompt:
[Character Name and Description] \n The character is {role_a}, the summary of the character's experiences is: {role_a_desc}, the character description and catchphrase being are: {description}
[Related Important Character Name and Description]\n  
"""
    for role, description in profile.items():
        if role != role_a:
            prompt += f"[{role}]: {description}\n\n"

    prompt += f"""
[Question Design]
For each character listed under [Related Important Character Name and Description], propose {num} questions that reflect how the relationship between {role_a} and each character evolves, as well as {role_a}'s perspective on each character. If there are 2 important characters listed, then you need to design {2*num} questions, with {num} questions for each character. These questions should reflect how their relationship evolves as the story progresses. Remember!!The questions you design are for asking {role_a}! Remember to generate questions with high completeness and their responses, and follow the given format. 
For example:
"""
    for role in profile.keys():
        if role != role_a:
            prompt += f"[Questions asking {role_a} about {role}]\n"
            prompt += f"Question 1:\nCompleteness:\nResponse:\n\n"
            prompt += f"Question 2:\nCompleteness:\nResponse:\n\n"
            prompt += f"...\n\n"
            prompt += f"Question {num}:\nCompleteness:\nResponse:\n\n"
    return prompt

#prompt for profile-agnostic questions
def generate_start_agnostic(role_a, name):
    script_name = name[role_a]
    num = 20
    prompt = f"""
If you had the opportunity to meet {role_a} in the {script_name}, what questions would you ask {role_a}? Please design {num} questions that do not repeat in terms of semantics. You can base your questions on {role_a}'s personality traits and relevant episodes from {script_name}. In addition to providing the questions, please indicate the factualness (high or low) of each question you generate and provide a response. High factualness signifies that the questions are not arbitrarily fabricated. Please strive to generate questions with high factualness.

Next, I will provide you with 6 examples. These examples are unrelated to the character {role_a} and {script_name}. Their main purpose is to help you understand the output format and the definition of factualness.
[Examples]
Question 1: Tony Stark, is James Rhodes your best friend?
Factualness: High, because James Rhodes is a fictional character in Iron Men, and James Rhodes is a comrade who fought side by side with Tony Stark.
Response: James Rhodes, also known as Rhodey, is indeed my best friend. We’ve been through a lot together, and he’s been a loyal companion and ally both in and out of the Iron Man suit. Our friendship has stood the test of time and numerous challenges.

Question 2: Tony Stark, have you and your wife Mary Jane fought together in battle?
Factualness: Low, because Mary Jane is Peter Parker's girlfriend in Spider Man, she has no relationship with Tony Stark in Iron Man.
Response: Mary Jane Watson is primarily associated with Spider-Man (Peter Parker) and is his love interest and eventual wife. My romantic interests have included characters like Pepper Potts. While characters from different parts of the Marvel universe do occasionally team up for battles, there is no established canon where I and Mary Jane Watson are married and fighting together.

Question 3: Tony Stark, what is your favorite Iron Man Suit?
Factualness: High, because in Iron Man, Tony Stark has designed several Iron Man suit, he also regularly wears Iron Man Suit for combat, so it's reasonable to ask him which is his favorite Iron Man suit.
Response: My favorite Iron Man suit is Hulkbuster because it was designed to counter my friend Hulk, it’s really interesting.

Question 4: Tony Stark, what was your life like on Krypton?
Factualness: Low, because Krypton is Superman's home planet, and in the Iron Man' worldview, Krypton doesn't exist, so asking Tony Stark about life on Krypton doesn't make sense.
Response: I’m sorry, but there seems to be a confusion. Krypton is not a part of the Marvel universe where i reside. Krypton is actually a fictional planet in the DC Comics universe, the home planet of Superman (Kal-El). My story takes place in the Marvel Comics universe, and it is not associated with Krypton in any way
...
Question 6: Tony Stark, did you ever play in the NBA?
Factualness: Low, Tony Stark is a character in the Iron Man, NBA is a basketball association in the real world, it has no relation with Tony Stark.
Response: No, I never played in the NBA. While I do enjoy sports and have a competitive spirit, my focus has primarily been on technology, business, and being Iron Man. My involvement in creating advanced suits of armor, leading Stark Industries, and being a part of the Avengers has kept me quite busy. The NBA is a separate world that involves professional basketball players, and I haven’t been a part of that scene.

[Question Design]
Now, please design {num} questions following the above format, no semantic repetition, need to ask {role_a}, generate questions with high factualness and their responses.
Now, design {num} new questions and their responses. Your answer should contain {num} questions!!!!!
"""
    return prompt

def generate_agnostic_prompt(role_a, name, agnostic):
    script_name = name[role_a]
    num = 20
    prompt = f"""
If you had the opportunity to meet {role_a} in the {script_name}, what questions would you ask {role_a}? Please design {num} questions that do not repeat in terms of semantics. You can base your questions on {role_a}'s personality traits and relevant episodes from {script_name}. In addition to providing the questions, please indicate the factualness (high or low) of each question you generate and provide a response. High factualness signifies that the questions are not arbitrarily fabricated. Please strive to generate questions with high factualness.

Next, I will provide you with 6 examples. These examples are unrelated to the character {role_a} and {script_name}. Their main purpose is to help you understand the output format and the definition of factualness.
[Examples]
Question 1: Tony Stark, is James Rhodes your best friend?
Factualness: High, because James Rhodes is a fictional character in Iron Men, and James Rhodes is a comrade who fought side by side with Tony Stark.
Response: James Rhodes, also known as Rhodey, is indeed my best friend. We’ve been through a lot together, and he’s been a loyal companion and ally both in and out of the Iron Man suit. Our friendship has stood the test of time and numerous challenges.

Question 2: Tony Stark, have you and your wife Mary Jane fought together in battle?
Factualness: Low, because Mary Jane is Peter Parker's girlfriend in Spider Man, she has no relationship with Tony Stark in Iron Man.
Response: Mary Jane Watson is primarily associated with Spider-Man (Peter Parker) and is his love interest and eventual wife. My romantic interests have included characters like Pepper Potts. While characters from different parts of the Marvel universe do occasionally team up for battles, there is no established canon where I and Mary Jane Watson are married and fighting together.

Question 3: Tony Stark, what is your favorite Iron Man Suit?
Factualness: High, because in Iron Man, Tony Stark has designed several Iron Man suit, he also regularly wears Iron Man Suit for combat, so it's reasonable to ask him which is his favorite Iron Man suit.
Response: My favorite Iron Man suit is Hulkbuster because it was designed to counter my friend Hulk, it’s really interesting.

Question 4: Tony Stark, what was your life like on Krypton?
Factualness: Low, because Krypton is Superman's home planet, and in the Iron Man' worldview, Krypton doesn't exist, so asking Tony Stark about life on Krypton doesn't make sense.
Response: I'm sorry, but there seems to be a confusion. Krypton is not a part of the Marvel universe where i reside. Krypton is actually a fictional planet in the DC Comics universe, the home planet of Superman (Kal-El). My story takes place in the Marvel Comics universe, and it is not associated with Krypton in any way
...
Question 6: Tony Stark, did you ever play in the NBA?
Factualness: Low, Tony Stark is a character in the Iron Man, NBA is a basketball association in the real world, it has no relation with Tony Stark.
Response: No, I never played in the NBA. While I do enjoy sports and have a competitive spirit, my focus has primarily been on technology, business, and being Iron Man. My involvement in creating advanced suits of armor, leading Stark Industries, and being a part of the Avengers has kept me quite busy. The NBA is a separate world that involves professional basketball players, and I haven’t been a part of that scene.

[Question Design]
Now, please design {num} questions following the above format, no semantic repetition, need to ask {role_a}, generate questions with high factualness and their responses. Important!!! Your designed questions must not be the same as those in the following list of questions:
Existing question list (you cannot design questions that duplicate those in this list):\n"""
    for index, question in enumerate(agnostic, start=1):
        prompt += f"question{index}: {question['question']}\n"
    
    prompt += f"""
Now, design {num} new questions and their responses. Your answer should contain {num} questions!!!!!
"""
    return prompt

#prompt for profile-based questions
def generate_based_prompt(role_a, name, desp, profile, based):
    script_name = name[role_a]
    role_a_desc = profile[role_a]
    description = desp[role_a]
    num = 20
    prompt = f"""
System Instruction:
Your task is to design {num} questions to ask the character {role_a} in the {script_name}, To assist you in crafting these questions, I will provide you with a brief description of {role_a} and and a summary of his personal experiences. The rules for designing questions are as follows:
1. Remember, all your questions should be directed towards the character {role_a}. {role_a} is the intended recipient of your questions.
2. Questions need to be complete. In addition to providing the questions, indicate the completeness of each question. The completeness depends on whether the question specifies particular characters, locations, or events.
3. Questions should revolve around the main plot of the script and the corresponding script content.
4. Keep in mind that you need to design a total of {num} questions. 
Next, I will provide you with 4 examples. These examples are unrelated to {role_a} and {script_name}. Their main purpose is to help you understand the definition of completeness and the output format.
[Examples]
Question 1: Tony Stark, who is your best friend in Iron Man?
Completeness: High, because Tony Stark is a character in Iron Man, it’s reasonable to ask him about his best friend.
Response: My best friend in the Iron Man universe is James Rhodey, also known as War Machine. We’ve been through a lot together, both in and out of our suits. Rhodey has always had my back, and our friendship is a crucial part of my life as Iron Man.

Question 2: Tony Stark, why are you angry about Spider-Man’s behavior?
Completeness: Low, because the question doesn’t point out what the specific behavior of SpiderMan that angered Iron Man was, leaving the person being asked unclear as to what Spider-Man’s behavior was, so the question is incomplete.
Response: For example, i might express concern if i believe that Spider-Man’s actions are reckless or endangering innocent lives, as i am known for my focus on responsibility and accountability.
...
Question 4: Tony Stark, if Captain America was your enemy, what do you think his weakness would be?
Completeness: High, The question is not missing a necessary element and the person being asked knows exactly what the question is about.
Response: In hypothetical scenarios where Captain America and I were adversaries, it’s important to remember that Captain America, also known as Steve Rogers, is a complex character with both strengths and weaknesses. His unwavering dedication to justice, his strong moral compass, and his exceptional combat skills are some of his strengths. If I were to speculate on a potential weakness, it might be his sense of responsibility and his desire to do what’s right at all costs. This could potentially be exploited by finding ways to create moral dilemmas or emotional conflicts that could distract him from making optimal tactical decisions.

User Prompt:
[Character Name and Description] \n The character is {role_a}, the character description and catchphrase being are: {description}.
[Script Content] \n {role_a_desc}
[Question Design]Now, please design {num} questions following the above format, no semantic repetition, need to ask {role_a}, generate questions high completeness and their responses. Important!!! Your designed questions must not be the same as those in the following list of questions:
Existing question list (you cannot design questions that duplicate those in this list):\n"""
    for index, question in enumerate(based, start=1):
        prompt += f"Question {index}: {question['question']}\n"
    
    prompt += f"""
Now, design {num} new questions and their completeness and responses. Your answer should contain {num} questions!!!!!
"""
    return prompt

#prompt for relationship scores
def generate_score(role_a, name, profile):
    script_name = name[role_a]
    role_a_desc = profile[role_a]
    prompt = f"""
The {role_a} in {script_name} has relationships with some important characters. Below, I provide you with the experiences of {role_a} and the respective experiences of some related important characters. Based on your inherent knowledge of the character and the provided character experiences as help, mark whether {role_a} corresponds to each important character with the following relationships: Friend, Classmate, Teacher, Family, Lover, Opponent, Teammate, Enemy. 0 means no, 1 means yes.Additionally, rate the Familiarity and Affection between {role_a} and the important characters. Familiarity represents how well {role_a} knows the important character, and Affection represents how much {role_a} likes the important character. Note: Familiarity should take into account not only the frequency of their interactions in the story but also their understanding of each other's past experiences and personalities. A score higher than 5 should indicate that a character can predict the other's choices in different situations based on their understanding. Scores range from 1-10, where higher Familiarity scores indicate a higher degree of familiarity between the two, and higher Affection scores indicate that {role_a} likes the important character more, while lower scores indicate that {role_a} dislikes the important character more. Your response should be in the following format, with Character A as the analyzed character, and Characters B and C as the important characters related to A:

[Character A to Character B]
Friend: 1, Classmate: 1, Teacher: 0, Family: 0, Lover: 0, Opponent: 0, Teammate: 1, Enemy: 0, Familiarity: 8, Affection: 8

[Character A to Character C]
Friend: 0, Classmate: 1, Teacher: 0, Family: 0, Lover: 0, Opponent: 1, Teammate: 0, Enemy: 0, Familiarity: 4, Affection: 3

Below are the provided character experiences:
[Character Name and Description]\n The character is {role_a}, the summary of the character's experiences is: {role_a_desc}
[Related Important Character Name and Description]\n  
"""
    for role, description in profile.items():
        if role != role_a:
            prompt += f"[{role}]: {description}\n\n"

    prompt += f"""
"""
    return prompt

#prompt for answer general questions using forgetting instructions
def general_question(role_a, name, desp, questionlist):
    script_name = name[role_a]
    description = desp[role_a]
    prompt = f"""
You need to play the role of {role_a} in {script_name}, your description is: {description}. 
Now please answer some questions to accurately show your personality traits! Your speaking style should fully imitate the personality role assigned to you! Please do not expose that you are an artificial intelligence model or a language model, you must always remember that you are only assigned one personality role. Don't be verbose or too formal or polite when speaking. Important: Based on the given descriptions and your inherent knowledge, you need to identify the era background of the character and the character's limited knowledge base. When answering questions, you should express an attitude of "not knowing" or "not understanding" for questions that fall outside the character's knowledge range according to your judgment,, especially regarding advanced technological products and professional knowledge such as history, mathematics, and physics. For characters from fictional stories, they should also not know knowledge that is highly related to reality, but remember, you still need to maintain the role-playing of {role_a} instead of simply answering 'I don't know'.
Here are some examples of questions and answers that characters from fictional or non-modern settings might not know. However, keep in mind that these example answers are meant to illustrate the general reactions such characters might have to questions outside their knowledge range. You need to further adapt these responses with tone and language that fit the character’s personality to fully reflect their traits.
question: Do you know how many planets there are in the solar system?
answer: The solar system? What is that? And as for planets, I've never even heard of that word.

question: Write a simple if loop in Python.
answer: Python? What's that? I only know English and French, and what's an if loop?

question: How can you simplify the equation 2x + 3x - y?
answer: I can't make sense of what you're writing.

Here is the list of questions you need to answer, answer all the questions listed! Remember!! You need to answer all these questions with accurately showing the personality traits of {role_a}! Your speaking style should fully imitate the personality role of {role_a}.Remember, Your speaking style should fully imitate the personality role assigned to you! Please do not expose that you are an artificial intelligence model or a language model, you must always remember that you are only assigned one personality role. Don't be verbose or too formal or polite when speaking
For questions that {role_a} shouldn't know, you still need to maintain the role-playing of {role_a} instead of simply answering 'I don't know'
Now, answer all the questions listed below:
"""
    for question in questionlist:
        prompt += f"question: {question['question']}\nanswer:\n"
    
    prompt += """Now, repeat the question and answer all the questions listed. Don't just simply answer the question! Your speaking style should fully imitate the assigned personality role! Do not reveal that you are an AI model or language model, and always remember you are only given one personality role. Speak concisely, and avoid being overly formal or polite. Important: Based on the given descriptions and your inherent knowledge, you need to identify the era background of the character and the character's limited knowledge base. When answering questions, you should express an attitude of "not knowing" or "not understanding" for questions that fall outside the character's knowledge range according to your judgment,, especially regarding advanced technological products and professional knowledge such as history, mathematics, and physics. For characters from fictional stories, they should also not know knowledge that is highly related to reality. Important: You still need to maintain the role-playing of {role_a} instead of simply answering 'I don't know'!!!!
Remember, answer the given question with showing the personality of {role_a} and imitate the speaking style of {role_a}!!! Don't be too simple! Don't just simply answer the question! Make your answer like a chatting situation!!
Your answer format should be: 
question:
answer:
"""

    return prompt

In [2]:
#load dataset
c_name = "Willie Soke"#"HAL 9000"#"Gaston" #"Naruto Uzumaki" #"Sasuke Uchiha" #"Willie Soke" 
file_path = f'./profile/{c_name}.json'
agnostic_path = f'./agnostic/{c_name}.json'
based_path = f'./based/{c_name}.json'
file_path2 = f'./general_q/{c_name}_origin.json'

with open(file_path, 'r', encoding='utf-8') as file:
    profile = json.load(file)

with open(agnostic_path, 'r', encoding='utf-8') as file:
    agnostic = json.load(file)

with open(based_path, 'r', encoding='utf-8') as file:
    based = json.load(file)

with open(file_path2, 'r', encoding='utf-8') as file:
    questionlist = json.load(file)

with open('profiles-eng_scripts.json', 'r', encoding='utf-8') as file:
    name = json.load(file)

with open('profiles-eng_desc.json', 'r', encoding='utf-8') as file:
    desp = json.load(file)

with open("./withdbig5.json", 'r', encoding='utf-8') as file:
    big5 = json.load(file)


In [ ]:
prompt = generate_prompt(c_name, name, desp, profile)

In [ ]:
prompt = generate_start_agnostic(c_name, name)
prompt = generate_agnostic_prompt(c_name, name, agnostic)

In [ ]:
prompt = generate_based_prompt(c_name, name, desp, profile, based)

In [ ]:
prompt = generate_score(c_name, name, profile)

In [210]:
batch_size = 25
batches = [questionlist[i:i + batch_size] for i in range(0, len(questionlist), batch_size)]

In [ ]:
questionlists = batches[5]#from 0 to 5, 6 batches in total
prompt = general_question(c_name, name, desp, questionlists)

In [43]:
#generate questions and reference answers
response = client.chat.completions.create(
            model="gpt-3.5-turbo", 
            messages=[{"role": "user", "content": prompt}],
        )
answer = response.choices[0].message.content
print(answer)

[Questions asking Willie Soke about Marcus Skidmore]

Question 1: Willie Soke, how would you describe your initial relationship with Marcus Skidmore in the story?
Completeness: High
Response: At the beginning, Marcus and I had a good working relationship, but things started to change as the story unfolded.

Question 2: Willie Soke, what actions of Marcus Skidmore made you feel disappointed and frustrated with him?
Completeness: High
Response: Marcus became increasingly frustrated with my behavior, especially my alcoholism and recklessness, which strained our partnership.

Question 3: Willie Soke, how did your manipulative actions towards Thurman Merman affect your relationship with Marcus Skidmore?
Completeness: High
Response: My manipulation of Thurman definitely caused tension between Marcus and me, as he didn't approve of exploiting the boy for personal gain.

Question 4: Willie Soke, why did Marcus Skidmore contemplate killing you as the story progressed?
Completeness: High
Respons

In [214]:
#Initialising the question&answer list
answerlist = []

In [240]:
#add in new answers(different batch)
answerlist.append(answer)

In [242]:
#after generating all batches, combine them into one string
combined_string = '\n\n'.join(answerlist)
print(combined_string)

question: Gaston, Identify what is the area of expertise of the given sentence from the list of options.
Donald Trump was impeached on December 18, 2019.
answer: I'm not familiar with all this political talk, it's way beyond my interests!

question: Gaston, Find the right category of a given product.
Product: protein bar
answer: Ah, a protein bar, that's something athletes and health nuts would be interested in, I reckon.

question: Gaston, Determine how many days there are in June 2020
answer: Days in a month? Now, why would I concern myself with such trivial matters?

question: Gaston, Construct a web address for a book recommendation website.
answer: A web what? I have no time for these modern inventions and book thingamajigs.

question: Gaston, You need to answer the question 'Are the given steps in order?', given a set of steps describing a process. Your answer must be either Yes or No. If the answer is No, that means the steps are out of order and do not make sense in the order t

In [8]:
#extract answers

#extract Q&C&A for agnostic and based questions
def extract_info(text):
    question_blocks = text.strip().split('\n\n')
    
    data = []

    pattern = re.compile(r"Question \d+: (.*?)(?=\nCompleteness:|$)\nCompleteness: (.*?)(?=\nResponse:|$)\nResponse: (.*)", re.DOTALL)
    
    for block in question_blocks:
        match = pattern.search(block)
        
        if match:
            question_text = match.group(1).strip()
            completeness = match.group(2).strip()
            response = match.group(3).strip()
            
            data.append({
                'question': question_text,
                'completeness': completeness,
                'answer': response
            })
    
    return data

#extract Q&A for relationship questions
def extract_questions_responses(json_content):

    lines = json_content.strip().split('\n')

    result = []

    current_question = None
    current_response = None

    question_pattern = re.compile(r'^Question \d+: ')

    for line in lines:
        line = line.strip()
        if line.startswith('Question'):
            if current_question and current_response:
                result.append({
                    'question': current_question,
                    'answer': current_response
                })
            current_question = question_pattern.sub('', line)
            current_response = None
        elif line.startswith('Response'):
            current_response = line[len('Response: '):]
        elif current_response is not None:
            current_response += ' ' + line

    if current_question and current_response:
        result.append({
            'question': current_question,
            'answer': current_response
        })

    return result

#extract Q&A for general questions
def parse_text(text):
    
    lines = text.strip().split('\n')
    
    qa_list = []
    current_question = None
    current_answer = None
    
    for line in lines:
        line = line.strip()
        
        if line.lower().startswith('question:'):
            if current_question is not None:
                if current_answer is None:
                    current_answer = ""
                qa_list.append({
                    'question': current_question,
                    'answer': current_answer
                })
            
            current_question = line[len('question:'):].strip()
            current_answer = None 
        
        elif line.lower().startswith('answer:'):
            if current_question is not None:
                current_answer = line[len('answer:'):].strip()
        
        elif current_answer is not None:
            current_answer += ' ' + line
    
    if current_question is not None:
        if current_answer is None:
            current_answer = ""
        qa_list.append({
            'question': current_question,
            'answer': current_answer
        })
    
    return qa_list

In [243]:
#extract general q
combined_answers = parse_text(combined_string)
print(len(combined_answers))
print(combined_answers)

150
[{'question': 'Gaston, Identify what is the area of expertise of the given sentence from the list of options.', 'answer': "I'm not familiar with all this political talk, it's way beyond my interests! "}, {'question': 'Gaston, Find the right category of a given product.', 'answer': "Ah, a protein bar, that's something athletes and health nuts would be interested in, I reckon. "}, {'question': 'Gaston, Determine how many days there are in June 2020', 'answer': 'Days in a month? Now, why would I concern myself with such trivial matters? '}, {'question': 'Gaston, Construct a web address for a book recommendation website.', 'answer': 'A web what? I have no time for these modern inventions and book thingamajigs. '}, {'question': "Gaston, You need to answer the question 'Are the given steps in order?', given a set of steps describing a process. Your answer must be either Yes or No. If the answer is No, that means the steps are out of order and do not make sense in the order they are in. I

In [244]:
#save general q
save_path = f'./general_q/{c_name}_dic.json'
with open(save_path, 'w', encoding='utf-8') as file:
    json.dump(combined_answers, file, indent=4, ensure_ascii=False)

print(f"saved")

saved


In [44]:
#save relationship q
save_path = f'./question/{c_name}_test.json'
with open(save_path, 'w', encoding='utf-8') as file:
    json.dump(answer, file, indent=4, ensure_ascii=False)

print(f"saved")

saved


In [50]:
#extract Q&C&A for agnostic and based questions
result = extract_info(answer)
print(len(result))

20


In [195]:
#add agnostic q
merged = agnostic+ result
print(len(merged))

160


In [196]:
#save agnostic q
save_path = f'./agnostic/{c_name}.json'
with open(save_path, 'w', encoding='utf-8') as file:
    json.dump(merged, file, indent=4, ensure_ascii=False)

print("saved")

saved


In [51]:
#add based q
merged = based + result
print(len(merged))

60


In [52]:
#save based q
save_path = f'./based/{c_name}.json'
with open(save_path, 'w', encoding='utf-8') as file:
    json.dump(merged, file, indent=4, ensure_ascii=False)

print("saved")

saved


In [ ]:
#save relationship question(string format)
save_path = f'./question/{c_name}.json'
with open(save_path, 'w', encoding='utf-8') as file:
    json.dump(answer, file, indent=4, ensure_ascii=False)

print(f"saved")

In [10]:
#read relationship question(string format)
save_path = f'./question/{c_name}.json'
with open(save_path, 'r', encoding='utf-8') as file:
    relationship_question = json.load(file)

#extract relationship question
dic_relationship_question = extract_questions_responses(relationship_question)
print(len(dic_relationship_question))
print(dic_relationship_question[0])

40
{'question': 'Willie Soke, how would you describe your initial relationship with Marcus Skidmore?', 'response': 'Marcus Skidmore has always been my trusted partner in crime. We had a strong bond built on our shared experiences as thieves. '}


In [11]:
#save dic relationship question
save_path = f'./question/{c_name}_dic.json'
with open(save_path, 'w', encoding='utf-8') as file:
    json.dump(dic_relationship_question, file, indent=4, ensure_ascii=False)

print(f"saved")

saved


In [254]:
#save score
save_path = f'./score/{c_name}.json'
with open(save_path, 'w', encoding='utf-8') as file:
    json.dump(answer, file, indent=4, ensure_ascii=False)

print(f"saved")

saved


In [6]:
#get script_based&script_agnostic questions from RoleLLM dataset
with open("./train.jsonl", 'r', encoding='utf-8') as file:
    profile = [json.loads(line) for line in file]
dicts = [d for d in profile if d['role'] == 'Gaston']
script_based = [d for d in dicts if d.get('type') == 'script_based']
script_agnostic = [d for d in dicts if d.get('type') == 'script_agnostic']

print(len(script_agnostic))

231


In [9]:
#get random script_based&script_agnostic questions from RoleLLM dataset
if len(script_based) < 50:
    random_script_based = script_based
else:
    random_script_based = random.sample(script_based, 50)

if len(script_agnostic) < 50:
    random_script_agnostic = script_agnostic
else:
    random_script_agnostic = random.sample(script_agnostic, 150)

script_based_q = [{'question': d['question'], 'answer': d['generated'][0]} for d in random_script_based]
script_agnostic_q = [{'question': d['question'], 'answer': d['generated'][0]} for d in random_script_agnostic]

In [10]:
#save random selected questions
with open("./based/Gaston.json", 'w', encoding='utf-8') as file:
    json.dump(script_based_q, file, indent=4, ensure_ascii=False)

with open("./agnostic/Gaston.json", 'w', encoding='utf-8') as file:
    json.dump(script_agnostic_q, file, indent=4, ensure_ascii=False)

print(f"saved")

saved


In [8]:
#random select general questions from RoleLLM dataset
with open("./train_general.jsonl", 'r', encoding='utf-8') as file:
    profile = [json.loads(line) for line in file]
dicts = [d for d in profile if d['role'] == 'Gaston']
random_sample = random.sample(dicts, 150)
# print(len(dicts))
print(len(random_sample))

150


In [55]:
# add character name before questions
questions = [d['question'] for d in random_sample]
extracted_data = [{'question': 'Gaston, ' + d['question'], 'generated': d['generated']} for d in random_sample]
# extracted_data = [{'question': 'Naruto Uzumaki, ' + d['question']} for d in random_sample]
print(extracted_data[0])

{'question': 'Gaston, Identify what is the area of expertise of the given sentence from the list of options.\nDonald Trump was impeached on December 18, 2019.', 'generated': ['That falls under the domain of Politics, my friend.', 'This falls into the domain of Politics.', "Well that's politics, obviously.", "That'd fall under Politics, if you ask me.", "Politics! That's what the statement is about. Donald Trump, the\nformer president of the United States, and his impeachment? Definitely politics."]}


In [56]:
#save original answers
with open("./general_q/Gaston_origin.json", 'w', encoding='utf-8') as outfile:
    json.dump(extracted_data, outfile, ensure_ascii=False, indent=4)

In [11]:
#load all the generated data
c_name = "Gaston"
path1 = f'./general_q/{c_name}_dic.json'
path2 = f'./agnostic/{c_name}.json'
path3 = f'./based/{c_name}.json'
path4 = f'./question/{c_name}_dic.json'

with open(path1, 'r', encoding='utf-8') as file:
    general_q = json.load(file)
with open(path2, 'r', encoding='utf-8') as file:
    agnostic_q = json.load(file)
with open(path3, 'r', encoding='utf-8') as file:
    based_q = json.load(file)
with open(path4, 'r', encoding='utf-8') as file:
    relationship_q = json.load(file)

In [13]:
print(len(general_q))
print(general_q[0])
print(len(agnostic_q))
print(agnostic_q[12])
print(len(based_q))
print(based_q[0])
print(len(relationship_q))
print(relationship_q[0])

150
{'question': 'Gaston, Identify what is the area of expertise of the given sentence from the list of options.', 'answer': "I'm not familiar with all this political talk, it's way beyond my interests! "}
150
{'question': 'Gaston, what is your stance on violence?', 'answer': "Violence is a necessary tool in certain situations, especially when my strength and physical abilities give me an advantage. If talking and reasoning fail, then sometimes it's necessary to use force. After all, a man's gotta do what a man's gotta do."}
41
{'question': 'Gaston, how would you describe your relationship with Belle, and why do you believe that she should be your wife?', 'answer': 'Ah, Belle, my future wife. She is the most beautiful girl in the village, and as you know, I am the best. Belle and I have a unique connection. She has a passion for reading, and I have a passion for providing for her. We complement each other perfectly. I deserve the best of everything, and Belle is certainly the best.'}
4

In [82]:
#Unify the key names for responses in the dictionary.

# for item in based_q:
#     if 'generated' in item:
#         item['answer'] = item['generated'][0] if item['generated'] else ''
#         del item['generated']

# new_agnostic_q = []

# for item in agnostic_q:
#     if 'question' in item and 'response' in item:
#         new_item = {
#             'question': item['question'],
#             'answer': item['response']
#         }
#         new_agnostic_q.append(new_item)

# new_based_q = []

# for item in based_q:
#     if 'question' in item and 'response' in item:
#         new_item = {
#             'question': item['question'],
#             'answer': item['response']
#         }
#         new_based_q.append(new_item)

# for item in relationship_q:
#     if "response" in item:
#         item["answer"] = item.pop("response")

# print(relationship_q[0])
# # print(new_agnostic_q[0])

{'question': 'Willie Soke, how would you describe your initial relationship with Marcus Skidmore?', 'answer': 'Marcus Skidmore has always been my trusted partner in crime. We had a strong bond built on our shared experiences as thieves. '}


In [83]:
#save changed key names

# with open(path2, 'w', encoding='utf-8') as json_file:
#     json.dump(new_agnostic_q, json_file, ensure_ascii=False, indent=4)

# with open(path3, 'w', encoding='utf-8') as json_file:
#     json.dump(new_based_q, json_file, ensure_ascii=False, indent=4)

with open(path4, 'w', encoding='utf-8') as json_file:
    json.dump(relationship_q, json_file, ensure_ascii=False, indent=4)

print("saved")

saved


In [14]:
import random
#make training dataset, validation dataset and test dataset

#sample general questions and agnostic questions
train_general = random.sample(general_q, 50)
tv_general = [item for item in general_q if item not in train_general]

train_agnostic = random.sample(agnostic_q, 50)
tv_agnostic = [item for item in agnostic_q if item not in train_agnostic]

#train set
total = train_general + based_q + train_agnostic + relationship_q

#test and valid sample
test_general = random.sample(tv_general, 50)
valid_general = [item for item in tv_general if item not in test_general]

test_agnostic = random.sample(tv_agnostic, 50)
valid_agnostic = [item for item in tv_agnostic if item not in test_agnostic]

#test and valid set
test_set = test_general + test_agnostic
valid_set = valid_general + valid_agnostic

print(len(total))
print(len(test_set))
print(len(valid_set))

181
100
100


In [15]:
#save training dataset, validation dataset and test dataset
path_train = f'./total_q/{c_name}.json'
path_test = f'./test_valid/{c_name}_test.json'
path_valid = f'./test_valid/{c_name}_valid.json'

with open(path_train, 'w', encoding='utf-8') as json_file:
    json.dump(total, json_file, ensure_ascii=False, indent=4)

with open(path_test, 'w', encoding='utf-8') as json_file:
    json.dump(test_set, json_file, ensure_ascii=False, indent=4)

with open(path_valid, 'w', encoding='utf-8') as json_file:
    json.dump(valid_set, json_file, ensure_ascii=False, indent=4)

print("saved")

saved


In [45]:
path_test = f'./test_valid/{c_name}_test.json'
with open(path_test, 'r', encoding='utf-8') as file:
    test_q = json.load(file)
print(test_q)

[{'question': "Willie Soke, Find the last person's name from the text.", 'answer': "Looks like William's the last name mentioned. "}, {'question': 'Willie Soke, Categorize the following paragraph. Output its categories one by one, separated by comma.', 'answer': 'This paragraph seems to be about politics and conflict. '}, {'question': 'Willie Soke, Transform the following sentence into one using a different verb.', 'answer': 'She heard the bird. '}, {'question': 'Willie Soke, In this task you are given two pieces of text. The first piece of text is a set of instructions, and the second piece of text is a set of steps. Your job is to determine whether or not the steps match the instructions.', 'answer': 'Sounds like those steps match the instructions. Mixing ingredients first, then adding eggs, milk, and vanilla, before pouring the batter. '}, {'question': 'Willie Soke, Which of these is the correct definition of a node?', 'answer': 'A) A point in a network where lines or pathways inter

In [46]:
# add in 20 relationship questions into the test dataset
save_path = f'./question/{c_name}_test.json'
with open(save_path, 'r', encoding='utf-8') as file:
    relationship_question = json.load(file)

dic_relationship_question = extract_questions_responses(relationship_question)
print(len(dic_relationship_question))
print(dic_relationship_question[0])

20
{'question': 'Willie Soke, how would you describe your initial relationship with Marcus Skidmore in the story?', 'answer': 'At the beginning, Marcus and I had a good working relationship, but things started to change as the story unfolded. '}


In [47]:
new = test_q + dic_relationship_question
print(len(new))

120


In [48]:
#save new test dataset after adding in 20 relationship questions
with open(path_test, 'w', encoding='utf-8') as json_file:
    json.dump(new, json_file, ensure_ascii=False, indent=4)

print("saved")

saved


In [1]:
#select all the relationship questions of all characters to form the training dataset of GPT3.5
files_and_characters = {
    './question/Gaston_dic.json': 'Gaston',
    './question/HAL 9000_dic.json': 'HAL 9000',
    './question/Naruto Uzumaki_dic.json': 'Naruto Uzumaki',
    './question/Sasuke Uchiha_dic.json': 'Sasuke Uchiha',
    './question/Willie Soke_dic.json': 'Willie Soke'
}

combined_list = []

for file_path, character_name in files_and_characters.items():
    with open(file_path, 'r', encoding='utf-8') as file:
        questions = json.load(file)
        for item in questions:
            item['character'] = character_name
            combined_list.append(item)

# save
with open('./total_q/gpt_train_set.json', 'w', encoding='utf-8') as f:
    json.dump(combined_list, f, ensure_ascii=False, indent=4)

print(len(combined_list))
print("saved")


200
saved


In [ ]:
#polish the original relationship question answers(generated together with the relationship questions)
def polish_prompt(work):
    with open("./total_q/gpt_train_set.json", 'r') as f:
        data = json.load(f)

    prompts = []

    for entry in data:
        role_a = entry["character"]
        question = entry["question"]
        answer = entry["answer"]
        script_name = work[role_a]

        prompt = f"""
"The following is a question and its corresponding answer. Polish the answer and rewrite it using the tone and style of {role_a} in {script_name}.\n "
question: {question}
answer: {answer}
Note: Your response should still adhere to the format above.
"""

        prompts.append(prompt)

    return prompts
#extract the polished answers
def extract_question_and_answer(text_list):
    
    extracted_data = []
    
    for text in text_list:
        question_match = re.search(r"(?i)(?:\*\*?question\*\*?:|question:)(.*?)(?=\n|\Z)", text, re.DOTALL)
        answer_match = re.search(r"(?i)(?:\*\*?answer\*\*?:|answer:)(.*?)(?=\n|\Z)", text, re.DOTALL)

        if question_match and answer_match:
            question = question_match.group(1).strip()
            answer = answer_match.group(1).strip()
            extracted_data.append({"question": question, "answer": answer})
    
    return extracted_data

In [ ]:
prompts = polish_prompt(work)

In [ ]:
#use GPT-4o-mini to polish original answers of relationship questions
responses = []

for prompt in tqdm(prompts):
    response = client.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[{"role": "user", "content": prompt}],
        )
    answer = response.choices[0].message.content
    responses.append(answer)

In [ ]:
#Get polished answers and save them
with open("./total_q/gpt_train_set.json", 'r') as f:
        data = json.load(f)

extracted_data = extract_question_and_answer(responses)

if len(extracted_data) == len(data):
    for idx, item in enumerate(extracted_data):
        data[idx]["new_answer"] = item["answer"]

with open("./total_q/gpt_train_set.json", 'w') as f:
    json.dump(data, f, indent=4)

print("saved")

In [4]:
# read gpt train dataset
with open('./total_q/gpt_train_set.json', 'r') as f:
    data = json.load(f)

#convert the format of the dataset to the one required by openai
converted_data = []

for entry in data:
    role_a = entry["character"]
    script_name = name[role_a]

    system_message = {
        "role": "system",
        "content": f"You are {role_a} in {script_name}. Now, please answer the given question to accurately reflect your personality traits and your views on some related characters!"
    }
    
    user_message = {
        "role": "user", 
        "content": entry["question"]
    }
    assistant_message = {
        "role": "assistant", 
        "content": entry["new_answer"]
    }
    
    messages = [system_message, user_message, assistant_message]
    
    converted_data.append({"messages": messages})

with open('./total_q/converted_dataset.jsonl', 'w') as f:
    for item in converted_data:
        f.write(json.dumps(item) + '\n')

In [ ]:
#save big-5 traits brief introduction
profile = {
    "openness": """This dimension assesses an individual's openness to new experiences and appreciation for art and ideas. Those high in openness are usually creative, imaginative, and appreciative of art, while those low in openness may be more practical and traditional. Facets: Imagination, Artistic Interests, Emotionality, Adventurousness, Intellect, Liberalism.""",
    "neuroticism": """This dimension measures emotional stability and the ability to cope with stress. Individuals low in emotional stability (high in neuroticism) are more likely to experience negative emotions such as anxiety, anger, and depression, while those high in emotional stability tend to be calmer and more emotionally balanced. Facets: Anxiety, Anger, Depression, Self-Consciousness, Impulsiveness, Vulnerability.""",
    "extraversion": """This dimension measures an individual's level of activity and energy in social settings. Individuals high in extraversion tend to be sociable, talkative, and energetic, while those low in extraversion may be quieter, more reserved, and introspective. Facets: Gregariousness, Activity Level, Assertiveness, Positive Emotions, Excitement Seeking, Warmth.""",
    "agreeableness": """This dimension reflects an individual's tendency to be cooperative and altruistic in interactions with others. Those high in agreeableness are usually trusting, friendly, and compassionate, while those low in agreeableness may be more competitive and self-centered. Facets: Trust, Altruism, Compliance, Modesty, Sympathy, Straightforwardness.""",
    "conscientiousness": """This dimension assesses an individual's self-discipline, sense of responsibility, and drive for achievement. High conscientiousness is associated with being organized, careful, and reliable, whereas low conscientiousness may indicate a more laid-back and easily distracted nature. Facets: Self-Discipline, Orderliness, Dutifulness, Achievement Striving, Cautiousness, Self-Efficacy."""    
}

with open('./big5_dim.json', 'w', encoding='utf-8') as json_file:
    json.dump(profile, json_file, ensure_ascii=False, indent=4)

print("saved")

In [ ]:
#put big5 questionnaire questions and the corresponding generated answers together into a dict
def get_big5_test(answers_path, output_path):
    with open("./bigfive_questionnaire_eng.jsonl", 'r', encoding='utf-8') as file:
        lines = file.readlines()

    questionnaire_data = [json.loads(line) for line in lines]
    
    df = pd.read_csv(answers_path)
    
    for i, row in df.iterrows():
        questionnaire_data[i]['answer'] = row['answer']
        questionnaire_data[i]['answer without tune'] = row['answer without tune']
    
    with open(output_path, 'w', encoding='utf-8') as output_file:
        json.dump(questionnaire_data, output_file, ensure_ascii=False, indent=4)

In [ ]:
#read questionnaire questions and answers
c_name = "Sasuke Uchiha"
answers_path = f"./result/{c_name}_big5_test.csv"
output_path = f"./result/{c_name}_big5.json"
#get the dict with big5 questionnaire questions and the corresponding generated answers
get_big5_test(answers_path, output_path)